In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
from matplotlib.patches import Circle
from collections import OrderedDict

In [3]:
# samwich routines
samwich_path = os.path.join(os.environ['HOME'],'waketracking')
if not samwich_path in sys.path:
    sys.path.append(samwich_path)
from samwich.dataloaders import XarrayData
from samwich.waketrackers import track

In [4]:
# case definition
from study import V27, neutral

# Animate wake tracking results from all methods

In [5]:
downstreamD = 6.0

## set up tracking parameters

In [6]:
case = neutral(casedir='WAKEBENCH-NREL_DEBNATH_LES/neutral',
               prefix='NREL_DEBNATH_LES_20190325',
               turbine=V27)
suffix = '--filtered'

In [7]:
Aref = case.turbine.rotor_area
D = case.turbine.D
zhub = case.turbine.zhub

In [8]:
from samwich.gaussian_functions import Bastankhah
gauss = Bastankhah(CT=case.turbine.CTref, d0=case.turbine.D, TI=case.TI/100)

Calculated k* : 0.0447339


## read all flowfield planes

In [9]:
%%time
data = XarrayData(case.get_wake_datafile(downstreamD),
                  uvar='U',vvar='V',wvar='W',
                  trim_time=case.trim_time(downstreamD))

Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_6D.nc
Calculated offset: 26
CPU times: user 1.5 s, sys: 1.45 s, total: 2.96 s
Wall time: 3.11 s


In [10]:
x,y,z,u = data.sliceI()

Slicing data at i=0 x=162.0


In [11]:
y1_cc = y[:,0]
y1 = np.concatenate((
    [y1_cc[0] - (y1_cc[1]-y1_cc[0])/2],
    (y1_cc[:-1] + y1_cc[1:])/2,
    [y1_cc[-1] + (y1_cc[-1]-y1_cc[-2])/2],
))

In [12]:
z1_cc = z[0,:]
z1 = np.concatenate((
    [z1_cc[0] - (z1_cc[1]-z1_cc[0])/2],
    (z1_cc[:-1] + z1_cc[1:])/2,
    [z1_cc[-1] + (z1_cc[-1]-z1_cc[-2])/2],
))

In [13]:
yy,zz = np.meshgrid(y1,z1,indexing='ij')

In [14]:
Nt = u.shape[0]
Nt

1578

## load all tracked planes

In [15]:
wake = OrderedDict()

In [16]:
def load_downstream_plane(downD,**tracking_params):
    name = tracking_params.pop('name')
    method = tracking_params.pop('method')
    # evaluate parameters that are a function of downstream distance
    for key,val in tracking_params.items():
        if isinstance(val, dict):
            tracking_params[key] = val[downD]
    print('x/D:',downD,':',tracking_params)
    # perform the wake tracking
    wo = track(data.sliceI(), method=method)
    wo.remove_shear(wind_profile=case.get_inflow(downD))
    yc,zc = wo.find_centers(**tracking_params,
                            **case.get_outputs(name,downD,suffix=suffix))
    return wo

### Gaussian 1D

In [17]:
Bastankhah_params = {
    'name': '1D Gaussian (Bastankhah)',
    'method': 'Gaussian',
    'umin': { x_D: gauss.amplitude(x_D*case.turbine.D,-case.Uref) for x_D in case.downstreamD },
    'sigma': { x_D: gauss.sigma(x_D*case.turbine.D) for x_D in case.downstreamD },
}

Calculated Gaussian width : 8.598824208554174 m
Calculated Gaussian amplitude : -4.474464240770766 m/s
Calculated Gaussian width : 9.806639508554174 m
Calculated Gaussian amplitude : -3.112149184268989 m/s
Calculated Gaussian width : 11.014454808554174 m
Calculated Gaussian amplitude : -2.340643747870897 m/s
Calculated Gaussian width : 12.222270108554174 m
Calculated Gaussian amplitude : -1.8396952310704735 m/s
Calculated Gaussian width : 13.430085408554174 m
Calculated Gaussian amplitude : -1.4902034362444538 m/s
Calculated Gaussian width : 14.637900708554174 m
Calculated Gaussian amplitude : -1.2345920413753313 m/s
Calculated Gaussian width : 15.845716008554174 m
Calculated Gaussian amplitude : -1.0410934863247117 m/s
Calculated Gaussian width : 8.598824208554174 m
Calculated Gaussian width : 9.806639508554174 m
Calculated Gaussian width : 11.014454808554174 m
Calculated Gaussian width : 12.222270108554174 m
Calculated Gaussian width : 13.430085408554174 m
Calculated Gaussian width :

In [18]:
%time wake['1D Gaussian (Bastankhah)'] = load_downstream_plane(downstreamD, **Bastankhah_params)

x/D: 6.0 : {'umin': -1.4902034362444538, 'sigma': 13.430085408554174}
Slicing data at i=0 x=162.0
Selected Tracker: Gaussian

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 162.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1578

...finished initializing WakeTracker

...finished initializing Gaussian 

  subtracting out time-varying profile
Trajectory loaded from ./WAKEBENCH-NREL_DEBNATH_LES/neutral/1D_Gaussian_Bastankhah--filtered/trajectory_6D.csv
Read pickled outlines from ./WAKEBENCH-NREL_DEBNATH_LES/neutral/1D_Gaussian_Bastankhah--filtered/outlines_6D.pkl
Note: wake tracking has already been performed
CPU times: user 2.15 s, sys: 313 ms, total: 2.47 s
Wall time: 1.38 s


### Gaussian 2D

In [19]:
Gaussian2D_params = {
    'name': '2D Gaussian',
    'method': 'Gaussian2D',
    'umin': None, # use VD minima in each snapshot
    'A_ref': Aref,
    'A_min': Aref/5,
    'A_max': 2*Aref,
    'AR_max': 10.0,
}

In [20]:
%time wake['2D Gaussian'] = load_downstream_plane(downstreamD, **Gaussian2D_params)

x/D: 6.0 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
Slicing data at i=0 x=162.0
Selected Tracker: Gaussian2D

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 162.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1578

...finished initializing WakeTracker

...finished initializing Gaussian2D 

  subtracting out time-varying profile
Trajectory loaded from ./WAKEBENCH-NREL_DEBNATH_LES/neutral/2D_Gaussian--filtered/trajectory_6D.csv
Read pickled outlines from ./WAKEBENCH-NREL_DEBNATH_LES/neutral/2D_Gaussian--filtered/outlines_6D.pkl
Note: wake tracking has already been performed
CPU times: user 2.05 s, sys: 312 ms, total: 2.36 s
Wall time: 1.17 s


### Constant area

In [21]:
const_area_params = {
    'name': 'Constant-Area Contours',
    'method': 'ConstantArea',
    'ref_area': Aref,
}

In [22]:
%time wake['Constant-Area Contours'] = load_downstream_plane(downstreamD, **const_area_params)

x/D: 6.0 : {'ref_area': 572.5552611167398}
Slicing data at i=0 x=162.0
Selected Tracker: ConstantArea

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 162.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1578

...finished initializing WakeTracker

...finished initializing ContourWakeTracker

...finished initializing ConstantArea 

  subtracting out time-varying profile
Trajectory loaded from ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Area_Contours--filtered/trajectory_6D.csv
Read pickled outlines from ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Area_Contours--filtered/outlines_6D.pkl
Note: wake tracking has already been performed
CPU times: user 2.1 s, sys: 316 ms, total: 2.42 s
Wall time: 1.32 s


### Constant flux
Recall: definition of velocity fields in SAMWICH Box is $u_{tot} = u_{avg} + u$

momentum flux $M = U_w(U_\infty - U_w) = -(U_w - U_\infty)U_w = $ `-u*u_tot`, for wake velocity $U_w$

In [23]:
def fluxfun(u,u_w):
    """x-momentum flux
    Note: function arguments correspond to `field_names` kwarg
    """
    return -u * u_w 

In [24]:
const_xmom_params = {
    'name': 'Constant-Xmom Contours',
    'method': 'ConstantFlux',
    'ref_flux': case.turbine.thrust(case.Uref),
    'flux_function': fluxfun,
    'field_names': ('u','u_tot'),
}

In [25]:
%time wake['Constant-Xmom Contours'] = load_downstream_plane(downstreamD, **const_xmom_params)

x/D: 6.0 : {'ref_flux': 13434.379391317067, 'flux_function': <function fluxfun at 0xd3b054598>, 'field_names': ('u', 'u_tot')}
Slicing data at i=0 x=162.0
Selected Tracker: ConstantFlux

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 162.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1578

...finished initializing WakeTracker

...finished initializing ContourWakeTracker

...finished initializing ConstantFlux 

  subtracting out time-varying profile
Trajectory loaded from ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours--filtered/trajectory_6D.csv
Read pickled outlines from ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Constant-Xmom_Contours--filtered/outlines_6D.pkl
Note: wake tracking has already been performed
CPU times: user 2.08 s, sys: 290 ms, total: 2.37 s
Wall time: 1.29 s


In [26]:
wake

OrderedDict([('1D Gaussian (Bastankhah)',
              Tracking 1578 sampled planes of vectors with shape (162,162)),
             ('2D Gaussian',
              Tracking 1578 sampled planes of vectors with shape (162,162)),
             ('Constant-Area Contours',
              Tracking 1578 sampled planes of vectors with shape (162,162)),
             ('Constant-Xmom Contours',
              Tracking 1578 sampled planes of vectors with shape (162,162))])

## plot

### total velocity

In [27]:
def plot_utot(itime,norm=D,colorset='tab10'):
    """Plot u"""
    wo = wake['1D Gaussian (Bastankhah)']  # doesn't matter which one we use
    fig,ax = plt.subplots(figsize=(8,6))
    ax.set_prop_cycle(cycler(color=plt.get_cmap(colorset).colors))
    pcm = ax.pcolormesh(yy/norm,(zz-zhub)/norm, wo.u_tot[itime,:,:]/case.Uref,
                        cmap='Greys_r', vmin=0.4, vmax=1.2)
    cbar = fig.colorbar(pcm, ax=ax)
#     rotor = Circle((0,0),radius=0.5, color='k', lw=1, fill=False)
#     ax.add_artist(rotor)
    ax.set_title('{:g} s'.format(itime),fontsize='x-large')
    cbar.set_label(r'$u / U_\infty$',fontsize='xx-large')
    cbar.ax.tick_params(labelsize='x-large')
    ax.tick_params(labelsize='x-large')
    ax.set_xlabel(r'$y/D$',fontsize='xx-large')
    ax.set_ylabel(r'$(z-z_{hub})/D$',fontsize='xx-large')
    ax.set_xlim((-1.5,1.5))
    return fig,ax

In [28]:
# %%time
# for i in range(Nt):
#     fig,ax = plot_utot(i)
#     fname = 'figures/utot_at_{:g}D_{:04d}.png'.format(downstreamD,i)
#     sys.stdout.write('\rGenerating '+fname)
#     fig.savefig(fname,bbox_inches='tight',dpi=300)
#     plt.close(fig)
# print('')

# Generating figures/utot_at_3D_1586.png
# CPU times: user 13min 45s, sys: 34.6 s, total: 14min 20s
# Wall time: 14min 23s

### wake velocity

In [29]:
def plot_wake(itime,norm=D,colorset='tab10'):
    """Plot u - uinf(t,z)"""
    fig,ax = plt.subplots(figsize=(8,6))
    ax.set_prop_cycle(cycler(color=plt.get_cmap(colorset).colors))
    cbar = None
    for name,wo in wake.items():
        if cbar is None:
            pcm = ax.pcolormesh(yy/norm,(zz-zhub)/norm, wo.u[itime,:,:]/case.Uref,
                                cmap='RdBu_r', vmin=-0.5, vmax=0.5)
            cbar = fig.colorbar(pcm, ax=ax)
        outline = wo.paths[itime]
        if outline is not None:
            ax.plot(outline[:,0]/norm,(outline[:,1]-zhub)/norm, lw=3, label=name)
    rotor = Circle((0,0),radius=0.5, color='k', lw=1, fill=False)
    ax.add_artist(rotor)
    ax.set_title('{:g} s'.format(itime),fontsize='x-large')
    cbar.set_label(r'$(u-U_\infty)/U_\infty$',fontsize='xx-large')
    cbar.ax.tick_params(labelsize='x-large')
    ax.tick_params(labelsize='x-large')
    ax.set_xlabel(r'$y/D$',fontsize='xx-large')
    ax.set_ylabel(r'$(z-z_{hub})/D$',fontsize='xx-large')
    ax.set_xlim((-1.5,1.5))
    ax.legend(loc='upper left', bbox_to_anchor=(1.2,1), fontsize='large')
    return fig,ax

In [30]:
%%time
for i in range(Nt):
    fname = 'figures/compare_wakes_at_{:g}D_{:04d}.png'.format(downstreamD,i)
    sys.stdout.write('\rGenerating '+fname)
    fig,ax = plot_wake(i)
    fig.savefig(fname,bbox_inches='tight',dpi=300)
    plt.close(fig)
print('')

# Generating figures/compare_wakes_at_3D_1586.png
# CPU times: user 16min 59s, sys: 51.5 s, total: 17min 51s
# Wall time: 17min 55s

Generating figures/compare_wakes_at_6D_1577.png
CPU times: user 16min 43s, sys: 38.1 s, total: 17min 21s
Wall time: 17min 25s


### wake velocity, in the MFoR

In [31]:
%%time
for name, wo in wake.items():
    wo.to_MFoR(y1_cc,z1_cc-zhub)

# CPU times: user 12 s, sys: 733 ms, total: 12.7 s
# Wall time: 12 s

Interpolating with RectBivariateSpline


Transform: frame 1577


Interpolating with RectBivariateSpline


Transform: frame 1577


Interpolating with RectBivariateSpline


Transform: frame 1577


Interpolating with RectBivariateSpline


Transform: frame 1577


CPU times: user 13.1 s, sys: 2.81 s, total: 15.9 s
Wall time: 17.8 s


In [39]:
def plot_wake_mfor(itime,norm=D,colorset='tab10'):
    """Plot u - uinf(t,z) in the meandering frame of reference"""
    fig,ax = plt.subplots(figsize=(8,6))
    ax.set_prop_cycle(cycler(color=plt.get_cmap(colorset).colors))
    cbar = None
    ylim = None
    for name,wo in wake.items():
        if cbar is None:
            pcm = ax.pcolormesh(yy/norm,(zz-zhub)/norm, wo.u_mfor[itime,:,:]/case.Uref,
                                cmap='RdBu_r', vmin=-0.5, vmax=0.5)
            cbar = fig.colorbar(pcm, ax=ax)
        if ylim is None:
            ylim = ax.get_ylim()
        outline = wo.paths_mfor[itime]
        if (outline is not None) and (len(outline) > 0):
            ax.plot(outline[:,0]/norm,outline[:,1]/norm, lw=3, label=name)
    rotor = Circle((0,0),radius=0.5, color='k', lw=1, fill=False)
    ax.add_artist(rotor)
    ax.set_title('{:g} s'.format(itime),fontsize='x-large')
    cbar.set_label(r'$(u-U_\infty)/U_\infty$',fontsize='xx-large')
    cbar.ax.tick_params(labelsize='x-large')
    ax.tick_params(labelsize='x-large')
    ax.set_xlabel(r'$y/D$',fontsize='xx-large')
    ax.set_ylabel(r'$(z-z_{hub})/D$',fontsize='xx-large')
    ax.set_xlim((-1.5,1.5))
    ax.set_ylim(ylim)
    ax.legend(loc='upper left', bbox_to_anchor=(1.2,1), fontsize='large')
    return fig,ax

In [40]:
%%time
for i in range(Nt):
    fname = 'figures/compare_wakes_mfor_at_{:g}D_{:04d}.png'.format(downstreamD,i)
    sys.stdout.write('\rGenerating '+fname)
    fig,ax = plot_wake_mfor(i)
    fig.savefig(fname,bbox_inches='tight',dpi=300)
    plt.close(fig)
print('')

# Generating figures/compare_wakes_mfor_at_3D_1586.png
# CPU times: user 16min 23s, sys: 35 s, total: 16min 58s
# Wall time: 17min 11s

Generating figures/compare_wakes_mfor_at_6D_1577.png
CPU times: user 15min 24s, sys: 1min 10s, total: 16min 34s
Wall time: 16min 40s
